In [14]:
# date: 25 march 2023
# author: bendlev
# purpose: Clean data for DIFUSE module, EEE 350

import pandas as pd

## Load and clean all variable data ##

**Load and clean passenger, flight, and loadfactor data**

In [15]:
i = 0

for var in ["Passengers", "Flights", "LoadFactor"]:

    var_name = var.lower()

    for carrier_name in ["Alaska", "American", "JetBlue", "Delta", "Hawaiian", "Southwest"]:

        cols_to_read = "A:E"

        path = r"C:\Users\bendl\Dropbox (Dartmouth College)\difuse-22f-statistics\data\raw\\" + var + "_10_16_2022 " + carrier_name + ".xlsx"

        df = pd.read_excel(path, sheet_name=0, header=1, usecols=cols_to_read, nrows=254) # nrows chosen to exclude null values at tail end of data

        # Rename/drop variables as necessary

        df = df.rename(columns={"Year": "year", "Month": "month", "TOTAL": var_name})

        df = df.drop(columns=['DOMESTIC', 'INTERNATIONAL'])

        # Drop rows with TOTAL in them, find them by identifying strings (months are typically int's)
        
        df = df[~df['month'].apply(lambda x: isinstance(x, str))]

        # limit universe to 2004-2021
        df = df[df['year'].apply(lambda x: x > 2003 and x < 2022)]

        # generate carrier_name variable for later merge

        if carrier_name == "Alaska":
            carrier = "Alaska Airlines Inc."
            carrier_path = carrier_name.lower()
        elif carrier_name == "American":
            carrier = "American Airlines Inc."
            carrier_path = carrier_name.lower()
        elif carrier_name == "JetBlue":
            carrier = "JetBlue Airways"
            carrier_path = carrier_name.lower()
        elif carrier_name == "Delta":
            carrier = "Delta Air Lines Inc."
            carrier_path = carrier_name.lower()
        elif carrier_name == "Southwest":
            carrier = "Southwest Airlines Co."
            carrier_path = carrier_name.lower()
        else:
            carrier = "Hawaiian Airlines Inc."
            carrier_path = carrier_name.lower()

        df["carrier_name"] = carrier

        df = df.reset_index(drop=True)

        df = df.loc[:, ['year', 'month', 'carrier_name', var_name]]

        # generate a variable whose value is a proportion relative to the first observation, to control for time change

        base_month = df.iloc[0, 3]

        var_name_percent = var_name + "_scaled"

        df[var_name_percent] = df[var_name] / base_month

        if i == 0:

            print(df)

            i += 1

        output_path = r"C:\Users\bendl\Dropbox (Dartmouth College)\difuse-22f-statistics\data\working_25march2023\\" + carrier_path + "_" + var_name + ".csv"

        df.to_csv(output_path, index=False)

        

     year month          carrier_name  passengers  passengers_scaled
0    2004     1  Alaska Airlines Inc.     1120840           1.000000
1    2004     2  Alaska Airlines Inc.     1132265           1.010193
2    2004     3  Alaska Airlines Inc.     1334725           1.190826
3    2004     4  Alaska Airlines Inc.     1289858           1.150796
4    2004     5  Alaska Airlines Inc.     1309469           1.168293
..    ...   ...                   ...         ...                ...
211  2021     8  Alaska Airlines Inc.     2429595           2.167656
212  2021     9  Alaska Airlines Inc.     2048281           1.827452
213  2021    10  Alaska Airlines Inc.     2256411           2.013143
214  2021    11  Alaska Airlines Inc.     2292449           2.045295
215  2021    12  Alaska Airlines Inc.     2318889           2.068885

[216 rows x 5 columns]


**Load and clean operating revenue data, slightly different than other forms of data**

In [16]:
i = 0

for carrier_name in ["Alaska", "American", "JetBlue", "Delta", "Hawaiian", "Southwest"]:

        cols_to_read = "A:H"

        path = r"C:\Users\bendl\Dropbox (Dartmouth College)\difuse-22f-statistics\data\raw\\" + "Operating_Rev" + "_10_16_2022 " + carrier_name + ".xlsx"

        df = pd.read_excel(path, sheet_name=0, header=1, usecols=cols_to_read, nrows=113) # nrows chosen to exclude null values at tail end of data

        # Rename/drop variables as necessary

        df = df.rename(columns={"Year": "year", "Quarter":"quarter", "TOTAL": "operating_rev"})

        df = df.drop(columns=['DOMESTIC', 'LATIN AMERICA', 'ATLANTIC', 'PACIFIC', 'INTERNATIONAL'])

        # Drop rows with Quarter equal to "TOTAL", identifying strings (where there should be int's)

        df = df[~df['quarter'].apply(lambda x: isinstance(x, str))]

        # limit universe to 2004-2021
        df = df[df['year'].apply(lambda x: x > 2003 and x < 2022)]

        # expand dataset by a factor of 3, then generate month variable

        df = df.loc[df.index.repeat(3)].reset_index(drop=True)

        # divide operating_rev by 3, since we have expanded dataset by 3 and assume that
        # the operating_rev is evenly split across each month

        df['operating_rev'] = df['operating_rev'] // 3

        df['month'] = (df.index % 12) + 1

        df = df.drop(columns=['quarter'])

        if carrier_name == "Alaska":
            carrier = "Alaska Airlines Inc."
            carrier_path = carrier_name.lower()
        elif carrier_name == "American":
            carrier = "American Airlines Inc."
            carrier_path = carrier_name.lower()
        elif carrier_name == "JetBlue":
            carrier = "JetBlue Airways"
            carrier_path = carrier_name.lower()
        elif carrier_name == "Delta":
            carrier = "Delta Air Lines Inc."
            carrier_path = carrier_name.lower()
        elif carrier_name == "Southwest":
            carrier = "Southwest Airlines Co."
            carrier_path = carrier_name.lower()
        else:
            carrier = "Hawaiian Airlines Inc."
            carrier_path = carrier_name.lower()

        df["carrier_name"] = carrier

        df = df.loc[:, ['year', 'month', 'carrier_name', 'operating_rev']]

        # generate a variable whose value is a proportion relative to the first observation, to control for time change

        base_month = df.iloc[0, 3]

        df["operating_rev_scaled"] = df["operating_rev"] / base_month

        if i == 0:
                
            print(df)

            i += 1

        output_path = r"C:\Users\bendl\Dropbox (Dartmouth College)\difuse-22f-statistics\data\working_25march2023\\" + carrier_path + "_" + "operatingrev" + ".csv"

        df.to_csv(output_path, index=False)

     year  month          carrier_name  operating_rev  operating_rev_scaled
0    2004      1  Alaska Airlines Inc.         163749              1.000000
1    2004      2  Alaska Airlines Inc.         163749              1.000000
2    2004      3  Alaska Airlines Inc.         163749              1.000000
3    2004      4  Alaska Airlines Inc.         192550              1.175885
4    2004      5  Alaska Airlines Inc.         192550              1.175885
..    ...    ...                   ...            ...                   ...
211  2021      8  Alaska Airlines Inc.         650479              3.972415
212  2021      9  Alaska Airlines Inc.         650479              3.972415
213  2021     10  Alaska Airlines Inc.         632408              3.862057
214  2021     11  Alaska Airlines Inc.         632408              3.862057
215  2021     12  Alaska Airlines Inc.         632408              3.862057

[216 rows x 5 columns]


## Merge all variables into a single dataset ##

### Step 1: Create collapsed files for each carrier ###

In [17]:
# Generate DataFrame that will be merged onto

# create a list of years
years = list(range(2004, 2022))

# create a list of months
months = list(range(1, 13))

# create a list of carriers
carriers = ['Alaska Airlines Inc.', 'American Airlines Inc.', 'JetBlue Airways', 'Delta Air Lines Inc.', 'Southwest Airlines Co.', 'Hawaiian Airlines Inc.']

# create a list of all possible year-month combinations
ym_combinations = [(y, m) for y in years for m in months]

# create a list of all possible carrier-year-month combinations
cym_combinations = [(c, y, m) for c in carriers for y, m in ym_combinations]

# create a list of dictionaries for each carrier-year-month combination
data = [{'year': y, 'month': m, 'carrier_name': c} for c, y, m in cym_combinations]

df = pd.DataFrame(data)

# generate carrier_code according to carrier_name

def get_carrier_code(carrier):
    carrier_dict = {"AS": 'Alaska Airlines Inc.',
                    "AA": 'American Airlines Inc.',
                    "DL": 'Delta Air Lines Inc.',
                    "HA": 'Hawaiian Airlines Inc.',
                    "B6": 'JetBlue Airways',
                    "WN": 'Southwest Airlines Co.'}
    for code, name in carrier_dict.items():
        if carrier == name:
            return code
    return None


df['carrier_code'] = df['carrier_name'].apply(get_carrier_code)

df = df.loc[:, ['year', 'month', 'carrier_code', 'carrier_name']]

# print(df)

# Individual carriers merged into their own datasets

read_path = r"C:\Users\bendl\Dropbox (Dartmouth College)\difuse-22f-statistics\data\working_25march2023\\"

for carrier in ['alaska', 'american', 'delta', 'hawaiian', 'jetblue', 'southwest']:

    merged = None

    # Read in the four files and store them in separate dataframes
    flights = pd.read_csv(read_path + '\\' + carrier + '_flights.csv')
    passengers = pd.read_csv(read_path + '\\' + carrier + '_passengers.csv')
    loadfactor = pd.read_csv(read_path + '\\' + carrier + '_loadfactor.csv')
    operatingrev = pd.read_csv(read_path + '\\' + carrier + '_operatingrev.csv')

    # merge all variables into the "merged" DataFrame
    merged = pd.merge(df, flights, on=['year', 'month', 'carrier_name'])
    merged = pd.merge(merged, loadfactor, on=['year', 'month', 'carrier_name'])
    merged = pd.merge(merged, operatingrev, on=['year', 'month', 'carrier_name'])
    merged = pd.merge(merged, passengers, on=['year', 'month', 'carrier_name'])

    # print(f"### {carrier} ###")
    # print(merged)

    merged = merged.rename(columns={'value_x': 'flights', 'value_y': 'passengers', 'value': 'loadfactor', 'value': 'operatingrev'})

    # print(merged)

    merged['pax_per_flight'] = merged['passengers'] / merged['flights']
    merged['pax_per_flight_scaled'] = merged['passengers_scaled'] / merged['flights_scaled']

    # print(merged)

    output_path = read_path + carrier + "_" + "collapsed.csv"

    # output to collapsed.csv, later to be appended into a single DataFrame
    merged.to_csv(output_path, index=False)

### Step 2: Concatenate the collapsed files in a single DataFrame ###

In [18]:
read_path = r"C:\Users\bendl\Dropbox (Dartmouth College)\difuse-22f-statistics\data\working_25march2023\\"

# List of carrier names
carriers = ["alaska", "american", "delta", "jetblue", "southwest", "hawaiian"]

# Initialize an empty dataframe to store the concatenated data
df_concatenated = pd.DataFrame()

# Loop through each carrier and read the corresponding CSV file
for carrier in carriers:
    file_path = read_path + f"{carrier}_collapsed.csv"
    df = pd.read_csv(file_path)

    # print(df)

    # Append the data to the concatenated dataframe
    df_concatenated = pd.concat([df_concatenated, df])

# Reset the index of the concatenated dataframe
df_concatenated = df_concatenated.reset_index(drop=True)

df_concatenated.to_csv(read_path + "airline_variables_by_month.csv", index=False)

## Merge collapsed dataset with delay variables ##

In [19]:
path = r"C:\Users\bendl\Dropbox (Dartmouth College)\difuse-22f-statistics\data\working_25march2023\\"

import numpy as np

{"AS": 'Alaska Airlines Inc.',
                    "AA": 'American Airlines Inc.',
                    "DL": 'Delta Air Lines Inc.',
                    "HA": 'Hawaiian Airlines Inc.',
                    "B6": 'JetBlue Airways',
                    "WN": 'Southwest Airlines Co.'}

filepath = path + "Airline_Delay_Cause_w-Percents.csv"

carrier_codes = ['AS', 'AA', 'DL', 'HA', 'B6', 'WN']

df = pd.read_csv(filepath, nrows=321430)

df = df[df['carrier'].apply(lambda x: x in carrier_codes)]

df = df[~df['year'].apply(lambda x: x < 2004 or x > 2021)]

df = df.sort_values('year')

df = df.reset_index(drop=True)

df = df.drop(columns=['Unnamed: 6'])

# print(df.columns.tolist())

vars = ['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'carrier_delay_percent', 'weather_delay_percent', 'nas_delay_percent', 'security_delay_percent', 'late_aircraft_delay_percent']

var_types = {var: int if 'percent' not in var else float for var in vars}

# vars = ['year', 'month', 'carrier', 'carrier_delay_percent', 'weather_delay_percent', 'nas_delay_percent', 'security_delay_percent', 'late_aircraft_delay_percent']

# Replace divide by zero errors and other unruly values with 0
df.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

#replace excel divide by 0 errors, specifically for percent columns
df = df.replace({'#DIV/0!': 0})

# set vartypes to numerics to work with .agg('mean') function
df = df.astype(var_types)

# aggregate by year, month, carrier, and take the mean of each col
df_means = df.groupby(['year', 'month', 'carrier']).agg('mean').reset_index()

# aggregate by year, month, carrier, and sum each column
df_sums = df.groupby(['year', 'month', 'carrier']).agg('sum').reset_index()

## ADJUST df_sums TO RECALCULATE PROPORTIONS

df_sums = df_sums.drop(columns=['weather_delay_percent', 'carrier_delay_percent', 'nas_delay_percent', 'security_delay_percent', 'late_aircraft_delay_percent'])

df_sums['total_delays'] = df_sums['weather_delay'] + df_sums['carrier_delay'] + df_sums['nas_delay'] + df_sums['security_delay'] + df_sums['late_aircraft_delay']

for delay_type in ['weather', 'carrier', 'nas', 'security', 'late_aircraft']:
    df_sums[delay_type + "_delay_percent"] = df_sums[delay_type + "_delay"] / df_sums['total_delays']

# print("DF Means")
# print(df_means)

df_sums.to_csv(path + "Airline_Delay_Cause_w-Sums.csv")

df_means.to_csv(path + "Airline_Delay_Cause_w-Percents_Means.csv")

# print(df.columns.tolist())

carrier_characteristics1 = pd.read_csv(path + "airline_variables_by_month.csv")

carrier_characteristics1 = carrier_characteristics1.rename(columns={'carrier_code': 'carrier'})

carrier_characteristics2 = pd.read_csv(path + "airline_variables_by_month.csv")

carrier_characteristics2 = carrier_characteristics2.rename(columns={'carrier_code': 'carrier'})

# print(df)

# print(carrier_characteristics)

new_df = pd.merge(df_means, carrier_characteristics1, on=['year', 'month', 'carrier'])

new_df2 = pd.merge(df_sums, carrier_characteristics2, on=['year', 'month', 'carrier'])

# print(new_df)

new_df = new_df.sort_values(['year', 'month', 'carrier'])

new_df2 = new_df2.sort_values(['year', 'month', 'carrier'])

carrier_name = new_df.pop('carrier_name')

carrier_name1 = new_df2.pop('carrier_name')

new_df.insert(loc=3, column='carrier_name', value=carrier_name)

new_df2.insert(loc=3, column='carrier_name', value=carrier_name1)

new_df.to_csv(path + "airline_variables_and_delays_by_month-means.csv", index=False)

new_df2.to_csv(path + "airline_variables_and_delays_by_month-sum.csv", index=False)

print(new_df)

      year  month carrier            carrier_name  carrier_delay  \
0     2004      1      AA  American Airlines Inc.    2548.793103   
1     2004      1      AS    Alaska Airlines Inc.    1062.456522   
2     2004      1      B6         JetBlue Airways     743.590909   
3     2004      1      DL    Delta Air Lines Inc.    1158.495327   
4     2004      1      HA  Hawaiian Airlines Inc.     655.500000   
...    ...    ...     ...                     ...            ...   
1291  2021     12      AS    Alaska Airlines Inc.    1252.013514   
1292  2021     12      B6         JetBlue Airways    3536.333333   
1293  2021     12      DL    Delta Air Lines Inc.    3391.992366   
1294  2021     12      HA  Hawaiian Airlines Inc.    1784.636364   
1295  2021     12      WN  Southwest Airlines Co.    4761.056075   

      weather_delay    nas_delay  security_delay  late_aircraft_delay  \
0       1129.080460  3797.632184       15.931034          3231.000000   
1        212.260870   635.217391     

C:\Users\bendl\AppData\Local\Temp\ipykernel_34852\465859436.py:46: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_means = df.groupby(['year', 'month', 'carrier']).agg('mean').reset_index()
C:\Users\bendl\AppData\Local\Temp\ipykernel_34852\465859436.py:49: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_sums = df.groupby(['year', 'month', 'carrier']).agg('sum').reset_index()
